# Indexação e Processamento de Consulta para a coleção de artigos obtidos no site Carta Capital

## Importando-se os pacotes necessários para a atividade

In [45]:
import pandas as pd
import re
import operator
import time
import sys
from collections import Counter
from unicodedata import normalize

## Obtendo-se coleção do site Carta Capital

In [46]:
data_frame = pd.read_csv("results.csv")
documentos = data_frame["text"].tolist()

### 1) Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

#### Algoritmo "parse" para obter os tokens de um determinado documento

In [47]:
def is_pontuacao(token):
    '''
        Retorna-se um booleano indicando se o token passado por parâmetro é uma pontuação.
    '''
    lista_pontuacoes = [",", ".", "!", "?", ":", "/", "#", "*", "(", ")", ";", ""]
    return token.strip() in lista_pontuacoes

def remove_acentuacoes(token):
    '''
        Remove as acentuações de um token.
    '''
    token_unicode = unicode(token, "utf-8")
    return normalize('NFKD', token_unicode).encode('ASCII','ignore')

def parse(texto):
    '''
        Transforma o texto em uma lista de tokens, eliminando espaços vazios, acentuações e pontuações, e
        tranforma todas as palavras em letras minúsculas.
    '''
    lista_texto = str(texto).split(" ")

    tokens = []
    for palavra in lista_texto:
        token = palavra.lower()
        token = token.strip()
        token = re.sub(r'[,.!?:*();]', '', token)
        token = remove_acentuacoes(token)
        
        if(not is_pontuacao(token)):
            tokens.append(token)
    
    return tokens

#### Algoritmo para obtenção dos índices invertidos

In [48]:
def get_indices_invertidos(documentos):
    '''
        Obtém uma lista de índices invertidos a partir de uma lista de documentos.
    '''
    indices_invertidos = {}
    for i in range(len(documentos)):
        doc = parse(documentos[i])
        documento_counter = Counter(doc)
        
        for indice in documento_counter.keys():
            if(indice in indices_invertidos):
                indices_invertidos[indice][i] = documento_counter[indice]
            else:
                indices_invertidos[indice] = { i: documento_counter[indice] }
    
    return indices_invertidos

#### Obtendo-se os índices invertidos a partir dos documentos obtidos do Carta Capital

In [50]:
indices_invertidos = get_indices_invertidos(documentos)

df_indices_invertidos = pd.DataFrame(pd.Series(indices_invertidos).reset_index()).set_axis(['Termo','Lista Invertida {documentID: Frequência}'],1,inplace=False)
df_indices_invertidos.set_index('Termo', inplace=True)
df_indices_invertidos.to_csv("indices_invertidos.csv")

### 2) Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18)

#### Algoritmo, implementado conforme é apresentado no livro texto, que executa a estratégia "documento por vez"

In [51]:
def documento_por_vez(consulta, k, indices_invertidos, documentos):
    L = []
    R = []

    for termo in consulta:
        li = indices_invertidos.get(termo)
        if li is not None:
            L.append(li)

    for d in range(1, len(documentos) + 1):
        score = 0
        for li in L:
            if d in li:
                score += li[d]

        if score > 0:
            R.append((score, d))

    R.sort(key = operator.itemgetter(0), reverse=True)

    return R[:k]

#### Algoritmo, implementado conforme é apresentado no livro texto, que executa a estratégia "termo por vez"

In [52]:
def termo_por_vez(consulta, k, indices_invertidos, documentos):
    L = []
    R = []

    for termo in consulta:
        li = indices_invertidos.get(termo)
        if li is not None:
            L.append(li)

    for d in range(1, len(documentos) + 1):
        score = 0
        for li in L:
            if d in li:
                score += li[d]

        if score > 0:
            R.append((score, d))

    R.sort(key = operator.itemgetter(0), reverse=True)
    
    return R[:k]

### 2.1) Defina 5 consultas de um termo somente.

### Palavras a serem consultadas
1. bolsonaro
2. brasil
3. casa
4. nome
5. vida

### 2.2) Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo)

In [53]:
print("Consulta para documento por vez")
print("\nConsulta para bolsonaro")
print(documento_por_vez(["bolsonaro"], 10, indices_invertidos, documentos))
print("\nConsulta para brasil")
print(documento_por_vez(["brasil"], 10, indices_invertidos, documentos))
print("\nConsulta para casa")
print(documento_por_vez(["casa"], 10, indices_invertidos, documentos))
print("\nConsulta para nome")
print(documento_por_vez(["nome"], 10, indices_invertidos, documentos))
print("\nConsulta para vida")
print(documento_por_vez(["vida"], 10, indices_invertidos, documentos))

print("\n\n")

print("Consulta para termo por vez")
print("\nConsulta para bolsonaro")
print(termo_por_vez(["bolsonaro"], 10, indices_invertidos, documentos))
print("\nConsulta para brasil")
print(termo_por_vez(["brasil"], 10, indices_invertidos, documentos))
print("\nConsulta para casa")
print(termo_por_vez(["casa"], 10, indices_invertidos, documentos))
print("\nConsulta para nome")
print(termo_por_vez(["nome"], 10, indices_invertidos, documentos))
print("\nConsulta para vida")
print(termo_por_vez(["vida"], 10, indices_invertidos, documentos))

Consulta para documento por vez

Consulta para bolsonaro
[(22, 425), (17, 338), (16, 12), (16, 31), (16, 355), (13, 168), (13, 348), (12, 79), (12, 374), (12, 383)]

Consulta para brasil
[(16, 104), (16, 141), (14, 361), (14, 430), (12, 374), (11, 77), (11, 144), (10, 48), (10, 102), (9, 114)]

Consulta para casa
[(11, 107), (7, 425), (4, 368), (3, 138), (3, 268), (3, 281), (3, 352), (3, 419), (2, 2), (2, 9)]

Consulta para nome
[(5, 68), (4, 112), (3, 158), (3, 301), (3, 324), (3, 406), (2, 18), (2, 125), (2, 176), (2, 224)]

Consulta para vida
[(6, 107), (5, 27), (5, 267), (4, 102), (4, 160), (4, 173), (4, 425), (3, 34), (3, 61), (3, 190)]



Consulta para termo por vez

Consulta para bolsonaro
[(22, 425), (17, 338), (16, 12), (16, 31), (16, 355), (13, 168), (13, 348), (12, 79), (12, 374), (12, 383)]

Consulta para brasil
[(16, 104), (16, 141), (14, 361), (14, 430), (12, 374), (11, 77), (11, 144), (10, 48), (10, 102), (9, 114)]

Consulta para casa
[(11, 107), (7, 425), (4, 368), (3, 

### 2.3) Dê evidências de que sua implementação está correta.

#### Obtendo todos os documentos que contém o termo bolsonaro

In [54]:
ids_documentos_bolsonaro = list(indices_invertidos["bolsonaro"].keys())

#### Verificando se a consulta documento por vez está correta

In [55]:
doc_por_vez_bolsonaro = documento_por_vez(["bolsonaro"], 10, indices_invertidos, documentos)

todos_ids_estao_doc_por_vez = all(doc[indice_id_termo] in ids_documentos_bolsonaro for doc in doc_por_vez_bolsonaro)

if(todos_ids_estao_doc_por_vez):
    print("Observe que todos os documentos que contém o termo bolsonaro foram encontrados na consulta de documento por vez!")
else:
    print("Isso demonstra que o algoritmo está errado, pois existem documentos encontrados na consulta que não possuem o termo bolsonaro!")

Observe que todos os documentos que contém o termo bolsonaro foram encontrados na consulta de documento por vez!


#### Verificando se a consulta termo por vez está correta

In [56]:
termo_por_vez_bolsonaro = termo_por_vez(["bolsonaro"], 10, indices_invertidos, documentos)

todos_ids_estao_termo_por_vez = all(doc[indice_id_termo] in ids_documentos_bolsonaro for doc in termo_por_vez_bolsonaro)

if(todos_ids_estao_termo_por_vez):
    print("Observe que todos os documentos que contém o termo bolsonaro foram encontrados na consulta de termo por vez!")
else:
    print("Isso demonstra que o algoritmo está errado, pois existem documentos encontrados na consulta que não possuem o termo bolsonaro!")

Observe que todos os documentos que contém o termo bolsonaro foram encontrados na consulta de termo por vez!


#### Conclusão

- As verificações demonstaram que ambos os algoritmos funcionam e que retornam os ids de documentos esperados.
- Outro ponto a se considerar é que ambos os algoritmos (documento por vez e termo por vez) retornam o mesmo resultado para a mesma consulta, mostrando a corretude de ambas as implementações.

### 2.4) Compare os tempos médios de execução e uso de memória de cada algoritmo.

#### Cálculo de tempo e memória para o algoritmo documento por vez

In [75]:
tempo_inicial_doc_por_vez = time.time()
resultado_doc_por_vez = documento_por_vez(["amigo"], 10, indices_invertidos, documentos)
tempo_final_doc_por_vez = time.time()
tempo_gasto_doc_por_vez = tempo_final_doc_por_vez - tempo_inicial_doc_por_vez

print("Tempo de execução para o documento por vez: %.15f" %tempo_gasto_doc_por_vez)
print("Memória consumida: %i" %sys.getsizeof(resultado_doc_por_vez))

Tempo de execução para o documento por vez: 0.000000000000000
Memória consumida: 144


#### Cálculo de tempo e memória para o algoritmo termo por vez

In [58]:
tempo_inicial_termo_por_vez = time.time()
resultado_tempo_por_vez = termo_por_vez(["amigo"], 10, indices_invertidos, documentos)
tempo_final_termo_por_vez = time.time()
tempo_gasto_termo_por_vez = tempo_final_termo_por_vez - tempo_inicial_termo_por_vez

print("Tempo de execução para o termo por vez: %.15f" %tempo_gasto_termo_por_vez)
print("Memória consumida: %i" %sys.getsizeof(resultado_tempo_por_vez))

Tempo de execução para o termo por vez: 0.000000000000000
Memória consumida: 144


#### Comparação entre os algoritmos

- Note que os algoritmos executam em um tempo muito próximo entre eles, sempre gastando um tempo na ordem de aproximadamente 10^-3 (dez elevado a menos três).
- Observe também que consomem extatamente a mesma memória, pois ambos utilizam duas lista auxiliares (L e R) e vão adicionando os mesmos termos à medida que são executados, sempre possuindo exatamente o mesmo fim para a mesma entrada.

### 3) Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21)

#### Implementação da consulta conjuntiva para documento por vez

In [59]:
def consulta_conjuntiva_documento_por_vez(consulta, k, indices_invertidos):
    L = []
    R = []

    for termo in consulta:
        li = indices_invertidos.get(termo)
        if li is not None:
            L.append(li)

    primeiro_em_indices_invertidos = L[0]

    for doc in primeiro_em_indices_invertidos:
        score = 0
        d = -1

        for li in L:
            if doc in li:
                d = doc
                score += li[doc]
            else:
                d = -1
                break

        if d > -1:
            R.append((score, doc))

    R.sort(key = operator.itemgetter(0), reverse=True)

    return R[:k]

### 3.1) Defina 5 consultas conjuntivas (AND).

#### Consultas:
1. presidente bolsonaro
2. luiz lula
3. brasil corrupcao
4. politica economica
5. casa civil

### 3.2) Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).

In [60]:
print("\nConsulta por presidente bolsonaro")
print(consulta_conjuntiva_documento_por_vez(["presidente", "bolsonaro"], 10, indices_invertidos))
print("\nConsulta por luiz lula")
print(consulta_conjuntiva_documento_por_vez(["luiz", "lula"], 10, indices_invertidos))
print("\nConsulta por brasil corrupcao")
print(consulta_conjuntiva_documento_por_vez(["brasil", "corrupcao"], 10, indices_invertidos))
print("\nConsulta para politica economica")
print(consulta_conjuntiva_documento_por_vez(["politica", "economica"], 10, indices_invertidos))
print("\nConsulta para casa civil")
print(consulta_conjuntiva_documento_por_vez(["casa", "civil"], 10, indices_invertidos))


Consulta por presidente bolsonaro
[(30, 425), (24, 3), (24, 338), (23, 12), (23, 31), (19, 348), (19, 355), (19, 373), (18, 383), (17, 76)]

Consulta por luiz lula
[(29, 407), (4, 425), (3, 385), (3, 410), (3, 365), (2, 83)]

Consulta por brasil corrupcao
[(18, 104), (14, 374), (11, 375), (8, 12), (7, 358), (6, 383), (5, 56), (5, 378), (5, 406), (4, 14)]

Consulta para politica economica
[(11, 329), (9, 358), (6, 374), (5, 204), (5, 370), (4, 229), (4, 256), (4, 334), (3, 8), (3, 24)]

Consulta para casa civil
[(12, 425), (4, 138), (4, 224), (4, 112), (3, 31), (3, 79), (3, 336), (2, 150), (2, 412), (2, 46)]


### 3.3) Dê evidências de que sua implementação está correta.

#### Construindo interseção entre os termos brasil e corrupcao.

In [61]:
ids_documentos_brasil = list(indices_invertidos["brasil"].keys())
ids_documentos_corrupcao = list(indices_invertidos["corrupcao"].keys())
intersecao_brasil_corrupcao = list(set(ids_documentos_brasil) & set(ids_documentos_corrupcao))

#### Consulta conjuntiva por brasil e corrupcao.

In [62]:
consulta_conjuntiva_brasil_corrupcao = consulta_conjuntiva_documento_por_vez(["brasil", "corrupcao"], 10, indices_invertidos)

#### Verificação da corretude do algoritmo

In [63]:
indice_id_termo = 1
todos_ids_estao_intersecao = all(elem[indice_id_termo] in intersecao_brasil_corrupcao for elem in consulta_conjuntiva_brasil_corrupcao)

if(todos_ids_estao_intersecao):
    print("Observe que todos os documentos obtidos em consulta_conjuntiva_brasil_corrupcao estão na interseção obtida em intersecao_brasil_corrupcao.")
    print("Isso dar uma evidência de que o algortimo funciona de fato!")
else:
    print("O algoritmo foi mal implementado, e nem todos os documentos estão na interseção!")

Observe que todos os documentos obtidos em consulta_conjuntiva_brasil_corrupcao estão na interseção obtida em intersecao_brasil_corrupcao.
Isso dar uma evidência de que o algortimo funciona de fato!
